# Projektarbeit Data Analytics

Sebastian Jana,
Sophie Jana

## Inhaltsverzeichnis
<a id ="inhaltsverzeichnis"></a>

[1. Aufgabe](#aufgabe1)

2. Aufgabe

3. Aufgabe

4. Aufgabe
5. Aufgabe
6. Aufgabe

[7. Quellenverzeichnis](#quellenverzeichnis)




### Aufgabe 1 (Datenvorbereitung)
<a id = "aufgabe1"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)

a) Einlesen der CSV Dateien, welche die Stromerzeugungsdaten und die Börsenstrompreise enthalten, als Dataframe df_hourly.


In [2]:
import pandas as pd
import numpy as np

In [ ]:
import os
import glob

path = './Daten/Strompreisdaten'
# List all files (.csv) in the given path
csv_files = glob.glob(os.path.join(path, "*csv"))

# df erzeugen
df_list = []
for i in range(len(csv_files)):
    try:
        df_temp = pd.read_csv(csv_files[i], sep = ",")
        for column in df_temp.columns:
            # Combine the date columns, by getting rid of the naming difference in csv source
            if 'Datum (MESZ)' in column:
                df_temp = df_temp.rename(columns = {'Datum (MESZ)':'Datum'})
            elif 'Datum (MEZ)' in column:
                df_temp = df_temp.rename(columns = {'Datum (MEZ)':'Datum'})
        df_list.append(df_temp)
    except Exception as e: 
        print("Fehler beim Einlesen des Files: ", csv_files[i])
    
df_hourly = pd.concat(df_list)
df_hourly

,Datum,Leistung nicht erneuerbar (MW),Leistung erneuerbar (MW),Day Ahead Auktion Preis (EUR/MWh)
0,2021-08-16 00:00:00,16015.81,19041.64,68.19
1,2021-08-16 01:00:00,15629.83,19420.11,65.03
2,2021-08-16 02:00:00,15322.51,19738.75,62.08
3,2021-08-16 03:00:00,15026.53,20822.64,60.13
4,2021-08-16 04:00:00,14987.63,21578.78,60.82
...,...,...,...,...
163,2021-04-25 19:00:00,18362.77,18593.89,65.93
164,2021-04-25 20:00:00,19477.05,16565.88,67.88
165,2021-04-25 21:00:00,19214.45,17819.31,66.00
166,2021-04-25 22:00:00,18964.11,18205.01,64.98


### Quellenverzeichnis
<a id = "quellenverzeichnis"></a>

[1] 

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)